This gets the configuration values set to either run it from a Windows or Mac environment.

In [41]:
WINDOWS_PROJECT_DIR = (
    "Y:\\garf\\Kris\\Projects\\NjCpaExpressWrongRenSolAppFix_20241211\\"
)


def windows():
    """
    Provides configuration settings for Windows environment.

    This function returns a dictionary containing various file paths and settings
    specific to the Windows operating system for the application.

    Returns:
        dict: A dictionary with the following keys:
            - send_list_path (str): Path to the CPA Express list file.
            - smtp_host (str): SMTP server host address.
            - applications_path (str): Path to the CPA applications directory.
            - signature_html_path (str): Path to the HTML signature file.
            - insured_plain_text_email_template_path (str): Path to the plain text email 
                template for insured.
            - co_producer_email_template_path (str): Path to the HTML email template 
                for co-producer.
            - insured_email_template_path (str): Path to the HTML email template for insured.
            - postgres_host (str): PostgreSQL database host address.
    """
    return {
        "send_list_path": WINDOWS_PROJECT_DIR + "CPA_Express_1022.txt",
        "smtp_host": "10.0.0.33",
        "applications_path": "Y:\\mailmerge\\CPA",
        "signature_html_path": WINDOWS_PROJECT_DIR + "signature.html",
        "insured_plain_text_email_template_path": WINDOWS_PROJECT_DIR
            + "insured_plain_text_email_template.txt",
        "co_producer_email_template_path": WINDOWS_PROJECT_DIR
            + "CoProducerEmailTemplate.html",
        "insured_email_template_path": WINDOWS_PROJECT_DIR
            + "InsuredEmailTemplate.html",
        "postgres_host": "localhost",
        "path_seperator": "\\"
    }


MAC_PROJECT_DIR = (
    "/Volumes/pdf_files/garf/Kris/Projects/NjCpaExpressWrongRenSolAppFix_20241211/"
)


def mac():
    """
    Provides configuration settings for Mac environment.

    This function returns a dictionary containing various file paths and settings
    specific to the Mac operating system for the application.

    Returns:
        dict: A dictionary with the following keys:
            - send_list_path (str): Path to the CPA Express list file.
            - smtp_host (str): SMTP server host address.
            - applications_path (str): Path to the CPA applications directory.
            - signature_html_path (str): Path to the HTML signature file.
            - insured_plain_text_email_template_path (str): Path to the plain text email 
                template for insured.
            - co_producer_email_template_path (str): Path to the HTML email template for 
                co-producer.
            - insured_email_template_path (str): Path to the HTML email template for insured.
            - postgres_host (str): PostgreSQL database host address.
    """
    return {
        "send_list_path": MAC_PROJECT_DIR + "CPA_Express_1022.txt",
        "smtp_host": "10.0.0.33",
        "applications_path": "/Volumes/pdf_files/mailmerge/CPA",
        "signature_html_path": MAC_PROJECT_DIR + "signature.html",
        "insured_plain_text_email_template_path": MAC_PROJECT_DIR
            + "insured_plain_text_email_template.txt",
        "co_producer_plain_text_email_template_path": MAC_PROJECT_DIR
            + "co_producer_plain_text_email_template.txt",
        "co_producer_email_template_path": MAC_PROJECT_DIR
            + "CoProducerEmailTemplate.html",
        "insured_email_template_path": MAC_PROJECT_DIR + "InsuredEmailTemplate.html",
        "postgres_host": "10.0.0.51",
        "path_seperator": "/"
    }

This is gets the variables and functions in place for the run

In [46]:
import smtplib
from email.message import EmailMessage
import os
import psycopg2
from psycopg2 import sql

config = mac()

db_params = {
    "host": config['postgres_host'],
    "database": "landy",
    "user": os.environ['LANDY_OPERATIONS_POSTGRES_USERNAME'],
    "password": os.environ['LANDY_OPERATIONS_POSTGRES_PASSWORD']
}

send_list_path = config['send_list_path']
smtp_host = config['smtp_host']
applications_path = config['applications_path']

signature_html = ''
co_producer_email_template = ''
insured_plain_text_email_template = ''
co_producer_plain_text_email_template = ''
insured_email_template = ''

with open(config['signature_html_path'], 'r') as signature_file, \
        open(config['co_producer_email_template_path'], 'r') as co_producer_email_template_file, \
        open(config['insured_plain_text_email_template_path'], 'r') as insured_plain_text_email_template_file, \
        open(config['co_producer_plain_text_email_template_path'], 'r') as co_producer_plain_text_email_template_file, \
        open(config['insured_email_template_path'], 'r') as insured_email_template_file:
    signature_html = signature_file.read()
    co_producer_email_template = co_producer_email_template_file.read()
    insured_plain_text_email_template = insured_plain_text_email_template_file.read()
    co_producer_plain_text_email_template = co_producer_plain_text_email_template_file.read()
    insured_email_template = insured_email_template_file.read()


def get_vars_from_row(values):
    insured_info = {}
    insured_info['insured_name'] = values[1].strip()
    insured_info['insured_email'] = values[35].strip()
    insured_info['expiration_date'] = values[37].strip()
    insured_info['producer_code'] = values[43].strip()
    insured_info['risk_code'] = values[45].strip()
    insured_info['policy_number'] = values[47].strip()
    insured_info['policy_number_digits'] = insured_info['policy_number'][3:-3]
    insured_info['term_year'] = insured_info['policy_number'][11:]
    return insured_info

def send_email(co_producer_email_template, co_producer_plain_text_email_template, s, insured_info, email, app_sub_folder):
    msg = EmailMessage()

    # generic email headers
    msg['Subject'] = f'RC: {insured_info['risk_code']}@{insured_info['term_year']} From Landy Insurance- Amended Renewal Solicitation Application ({insured_info['insured_name']} {insured_info['policy_number']})'
    msg['From'] = 'landy_insurance@landy.com'
    msg['Bcc'] = 'msg@landy.com'
    msg['To'] = email

    # set the plain text body
    msg.set_content(co_producer_plain_text_email_template.format(insured_name=insured_info['insured_name'], policy_number=insured_info['policy_number'], expiration_date=insured_info['expiration_date']))

    # set an alternative html body
    msg.add_alternative(co_producer_email_template.format(insured_name=insured_info['insured_name'], policy_number=insured_info['policy_number'], expiration_date=insured_info['expiration_date'], signature_html=signature_html), subtype='html')
                
    # \D44200_1022_A_Express_App
    file_path = f'{applications_path}{config['path_seperator']}{app_sub_folder}{config['path_seperator']}{insured_info['producer_code']}_{insured_info['policy_number']}.pdf'
    msg.add_attachment(open(file_path, 'rb').read(), 
                            filename=f"Renewal-Application-{insured_info['policy_number_digits']}.pdf", 
                            maintype='application/pdf',
                            subtype='mixed')

    # send the message via our own SMTP server
    s.send_message(msg)



This emails all of the accounts in the send list

In [ ]:

with open(send_list_path, 'r') as f, \
        smtplib.SMTP(smtp_host) as s, \
        psycopg2.connect(**db_params) as conn:
    for line in f:
        values = line.split("!")
        if len(values) > 2 and values[1] != 'contact' and values[1] != 'Accountants Renewal  0':
            insured_info = get_vars_from_row(values)
            if insured_info['producer_code'] != "HHL01-A":
                # Create a cursor object
                cur = conn.cursor()

                # Execute a query
                query = sql.SQL(f"SELECT \"email[1]\" FROM cpa_coprod WHERE cpcode = '{insured_info['producer_code']}';")
                cur.execute(query)
                producer_email = cur.fetchone()[0]

                print(f"Sending co-producer email to {insured_info['producer_code']} ({insured_info['producer_email']})")

                send_email(co_producer_email_template, co_producer_plain_text_email_template, s, insured_info, producer_email, "D44200_1022_A_Express_App")

            print(f"Sending email to insured {insured_info['risk_code']} - {insured_info['insured_name']} ({insured_info['insured_email']})")

            send_list_path = 'D44200_1022_A_Express_App'
            if insured_info['producer_code'] == 'HHL01-A':
                send_list_path = 'D44200_1022_D_Express_App'

            send_email(insured_email_template, insured_plain_text_email_template, s, insured_info, insured_info['insured_email'], send_list_path)

For testing the email send method

In [47]:
# values = 'TAKE LINE FROM SEND LIST'.split("!")
insured_info = get_vars_from_row(values)
producer_email = 'FILL IN ACTUAL EMAIL'
# Test before send
# producer_email = 'kristofer@landy.com'
with smtplib.SMTP(smtp_host) as s:
    send_email(co_producer_email_template, co_producer_plain_text_email_template, s, insured_info, producer_email, "D44200_1022_A_Express_App")